# Capstone 2 Modeling

In [7]:
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
#import warnings
#warnings.filterwarnings('ignore')



In [8]:
codon_list = ['UUU', 'UUC', 'UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG',
       'AUU', 'AUC', 'AUA', 'AUG', 'GUU', 'GUC', 'GUA', 'GUG', 'GCU', 'GCC',
       'GCA', 'GCG', 'CCU', 'CCC', 'CCA', 'CCG', 'UGG', 'GGU', 'GGC', 'GGA',
       'GGG', 'UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC', 'ACU', 'ACC', 'ACA',
       'ACG', 'UAU', 'UAC', 'CAA', 'CAG', 'AAU', 'AAC', 'UGU', 'UGC', 'CAU',
       'CAC', 'AAA', 'AAG', 'CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG', 'GAU',
       'GAC', 'GAA', 'GAG', 'UAA', 'UAG', 'UGA']

amino_list = ['alanine', 'arginine',
       'asparagine', 'aspartic acid', 'cysteine', 'glutamine', 'glutamic acid',
       'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
       'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan',
       'tyrosine', 'valine', 'start', 'stop']

In [9]:
dnacols = ['D_chloroplast', 'D_genomic', 'D_mitochondrial']
kingcols = ['K_bacteria', 'K_virus', 'K_plant', 'K_vertebrate', 'K_invertebrate',
            'K_mammal', 'K_bacteriophage', 'K_rodent', 'K_primate', 'K_archaea']


In [10]:
# importing the csv from the preprocessing notebook
cu = pd.read_csv('codon_usage3.csv')
cu.head()

,Unnamed: 0,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,...,K_invertebrate,K_mammal,K_plant,K_primate,K_rodent,K_vertebrate,K_virus,D_chloroplast,D_genomic,D_mitochondrial
0,0,100217,-0.108063,Epizootic haematopoietic necrosis virus,-0.468279,-0.984617,-0.973734,-1.141636,-0.546242,0.942351,...,0,0,0,0,0,0,1,0,1,0
1,1,100220,-0.108785,Bohle iridovirus,0.139840,-0.851972,-0.964991,-0.788651,-1.297120,0.696126,...,0,0,0,0,0,0,1,0,1,0
2,2,100755,-0.104089,Sweet potato leaf curl virus,-0.284696,-0.143098,-0.338885,0.145086,-0.943377,-0.495909,...,0,0,0,0,0,0,1,0,1,0
3,3,100880,-0.108174,Northern cereal mosaic virus,-0.398861,-0.087111,-0.211624,-0.449699,-0.202875,-0.326500,...,0,0,0,0,0,0,1,0,1,0
4,4,100887,-0.079183,Soil-borne cereal mosaic virus,0.198357,-0.839913,-0.625466,2.450823,-0.379275,-0.882051,...,0,0,0,0,0,0,1,0,1,0


To start, let's use the codons as predictors, and DNAtype as outcome, with k-nearest neighbors.


In [11]:
y = cu[dnacols]
X = cu[codon_list]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1331)

In [12]:
# considering values of hyperparameter k 
test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

for i in range(len(test_scores)):
    print("K =", i+1, "train score:", train_scores[i], "test score:", test_scores[i])

K = 1 train score: 0.9998897950187349 test score: 0.9940858832604783
K = 2 train score: 0.9948203658805378 test score: 0.9897145795834404
K = 3 train score: 0.9973550804496363 test score: 0.9933144767292363
K = 4 train score: 0.994269340974212 test score: 0.9910002571355104
K = 5 train score: 0.9958122107119242 test score: 0.9920287991771664
K = 6 train score: 0.9936081110866212 test score: 0.9907431216250965
K = 7 train score: 0.9950407758430682 test score: 0.9912573926459244
K = 8 train score: 0.9933877011240908 test score: 0.9892003085626125
K = 9 train score: 0.9943795459554772 test score: 0.9907431216250965
K = 10 train score: 0.9926162662552347 test score: 0.9889431730521985
K = 11 train score: 0.9931672911615606 test score: 0.9897145795834404
K = 12 train score: 0.9915142164425832 test score: 0.9876574955001286
K = 13 train score: 0.9925060612739696 test score: 0.9894574440730265
K = 14 train score: 0.9912938064800529 test score: 0.9879146310105426


The overall accuracies given above seem suspiciously high.  I feel like I need to do some kind of model diagnostics.  Let's consider what we get using the aminos instead of the codons.

In [13]:
X = cu[amino_list]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13131)

In [14]:
# considering values of hyperparameter k 
test_scores = []
train_scores = []

for i in range(1,11):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

for i in range(len(test_scores)):
    print("K =", i+1, "train score:", train_scores[i], "test score:", test_scores[i])

K = 1 train score: 0.9998897950187349 test score: 0.9922859346875803
K = 2 train score: 0.9905223716111968 test score: 0.9853432759064027
K = 3 train score: 0.9949305708618029 test score: 0.9935716122396503
K = 4 train score: 0.9907427815737271 test score: 0.9915145281563383
K = 5 train score: 0.9933877011240908 test score: 0.9922859346875803
K = 6 train score: 0.9911836014987877 test score: 0.9915145281563383
K = 7 train score: 0.9922856513114393 test score: 0.9930573412188223
K = 8 train score: 0.9899713467048711 test score: 0.9907431216250965
K = 9 train score: 0.9907427815737271 test score: 0.9920287991771664
K = 10 train score: 0.9890897068547498 test score: 0.9889431730521985


Again, all of these scores seem very high indeed.  Also note that here, unlike in the preceding case, some of the test scores are higher than the train scores.  This is a sure sign that something has gone glaringly wrong. What's happening here?  What if we try to model the kingdom (rather than DNAtype, as in the preceding instances) using this method?


In [16]:
y = cu[kingcols]
X = cu[amino_list]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=212)

In [17]:
test_scores = []
train_scores = []

for i in range(1,11):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

for i in range(len(test_scores)):
    print("K =", i+1, "train score:", train_scores[i], "test score:", test_scores[i])

K = 1 train score: 1.0 test score: 0.8675752121367961
K = 2 train score: 0.8642274630813312 test score: 0.780920545127282
K = 3 train score: 0.910072735287635 test score: 0.8434044741578812
K = 4 train score: 0.8442803614723385 test score: 0.7945487271792234
K = 5 train score: 0.8749173462640512 test score: 0.8302905631267679
K = 6 train score: 0.8288516640952172 test score: 0.7888917459501157
K = 7 train score: 0.8547498346925281 test score: 0.8138338904602725
K = 8 train score: 0.8160678862684594 test score: 0.7801491385960401
K = 9 train score: 0.8363456028212475 test score: 0.8079197737207509
K = 10 train score: 0.8009698038351334 test score: 0.7796348675752122


This is interesting - these results aren't suspiciously high.  Indeed, the k-NN models applied to kingdom as outcome have scores that look a little weak.  What if we use the codons (instead of aminos) to predict?


In [19]:
X = cu[codon_list]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2102)

In [20]:
test_scores = []
train_scores = []

for i in range(1,11):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

for i in range(len(test_scores)):
    print("K =", i+1, "train score:", train_scores[i], "test score:", test_scores[i])

K = 1 train score: 1.0 test score: 0.9362303934173309
K = 2 train score: 0.9300198368966277 test score: 0.8889174595011571
K = 3 train score: 0.9555873925501432 test score: 0.9244021599382874
K = 4 train score: 0.915803394313423 test score: 0.8886603239907431
K = 5 train score: 0.9355300859598854 test score: 0.913088197480072
K = 6 train score: 0.9042318712805819 test score: 0.8835176137824634
K = 7 train score: 0.9207626184703549 test score: 0.9004885574697866
K = 8 train score: 0.8923297333039454 test score: 0.8768320905116996
K = 9 train score: 0.9051135111307032 test score: 0.890460272563641
K = 10 train score: 0.883403129821468 test score: 0.870917973772178


Interestingly, I'm not seeing the same problems when fitting kNN to model 'kingdom' that I did modeling 'DNAtype' - what explains this?  If there were some problem with the train test split, or normalization, wouldn't it impact both sets of models?  This is something that I really need to explain.